In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [5]:
dataset = pd.read_csv('dataset.csv')

In [6]:
dataset

,custom_id,text,text_cleaned,domain,label
0,pla063804pla,"карта ашылмайды! сондықтан, әзірге 2 жұлдыз. к...",карта ашылмайды сондықтан әзірге 2 жұлдыз кейі...,appstore,1
1,kit003073kit,Керемет мотивация мен жаңа көзқарас қалыптасты...,керемет мотивация мен жаңа көзқарас қалыптасты...,bookstore,4
2,pla040287pla,"Дұрыс жұмыс жасамайды. Аймақты басасың, басқа ...",дұрыс жұмыс жасамайды аймақты басасың басқа жа...,appstore,0
3,kit005410kit,Соғыс кезіндегі халық көрген тауқыметті өз көз...,соғыс кезіндегі халық көрген тауқыметті өз көз...,bookstore,4
4,map001672map,Жетысу саябағы өте керемет екен. Ұнағаны шағын...,жетысу саябағы өте керемет екен ұнағаны шағын ...,mapping,4
...,...,...,...,...,...
506505,pla109872pla,МАШААЛЛА! Алла разы болсын!..,машаалла алла разы болсын,appstore,3
506506,fli001304fli,"Сапалы тігілген шалбар. Өлшемі тура, 48 размер...",сапалы тігілген шалбар өлшемі тура 48 размерге...,market,3
506507,kas005978kas,"Сақина өте әдемі, әпкеме алдым. Ол кісіге де ұ...",сақина өте әдемі әпкеме алдым ол кісіге де ұна...,market,3
506508,pla106012pla,Хабарлама келмей тұр,хабарлама келмей тұр,appstore,3


In [7]:
target = dataset['label'].values.tolist() #sutunu pandas serisi olarak aldık ve değerlerini liste olarak topladık
data = dataset['text_cleaned'].values.tolist()

data setini eğitim ve test seti olarak ayırmamız gerekiyor
eğitim seti üzerinde modeli eğitiyoruz ve test seti üzerind model çalışyor mu kotrol ediyoruz. 
eğitim data setinin %80 i test ise %20si


In [8]:
cutoff= int(len(data)*0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [9]:
x_train[500]

'жақсы екен әдемі'

In [10]:
y_train[800]

4

In [11]:
#tokenleştirme yaptık 
num_words=10000
tokenizer = Tokenizer(num_words=num_words) #kelime sınırı belirlendi 

In [12]:
tokenizer.fit_on_texts(data)

In [13]:
tokenizer.word_index

{'өте': 1,
 'жақсы': 2,
 'керемет': 3,
 'екен': 4,
 'маған': 5,
 'мен': 6,
 'ұнады': 7,
 'ойын': 8,
 'бірақ': 9,
 'неге': 10,
 'жоқ': 11,
 'бұл': 12,
 'емес': 13,
 'деп': 14,
 'күшті': 15,
 'керек': 16,
 'көп': 17,
 'бар': 18,
 'қатты': 19,
 'болады': 20,
 'бір': 21,
 'және': 22,
 'рахмет': 23,
 'не': 24,
 'еді': 25,
 'да': 26,
 'жаман': 27,
 'осы': 28,
 'жаксы': 29,
 'үшін': 30,
 'дұрыс': 31,
 'қазақ': 32,
 'тез': 33,
 'оте': 34,
 'нашар': 35,
 'сол': 36,
 'жатыр': 37,
 'тек': 38,
 'береді': 39,
 'деген': 40,
 'болды': 41,
 'сапасы': 42,
 'енді': 43,
 'тұр': 44,
 'болса': 45,
 'де': 46,
 'ыңғайлы': 47,
 'болар': 48,
 'ол': 49,
 '1': 50,
 'болып': 51,
 'қалай': 52,
 '2': 53,
 'жұлдыз': 54,
 'басқа': 55,
 'бәрі': 56,
 'маган': 57,
 'жұмыс': 58,
 'болсын': 59,
 'ғой': 60,
 'менің': 61,
 'алып': 62,
 'қызық': 63,
 '5': 64,
 'оиын': 65,
 'қазақша': 66,
 '3': 67,
 'ұнайды': 68,
 'унады': 69,
 'жок': 70,
 'ұнамады': 71,
 'менде': 72,
 'ма': 73,
 'ал': 74,
 'қайта': 75,
 'тамаша': 76,
 'кезде

In [14]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [15]:
x_train[800]

'аллаһ разы болсын өте тиімді программ а'

In [16]:
print(x_train_tokens[800])

[484, 85, 59, 1, 406, 215]


In [17]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [18]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [19]:
np.mean(num_tokens)

7.681698288286509

In [20]:
np.max(num_tokens)

226

In [21]:
np.argmax(num_tokens)

71648

In [22]:
x_train[21941]

'бұл ойын маған қатты ұнады әсерлі ойын екен'

In [23]:
max_tokens=np.mean(num_tokens)+2*np.std(num_tokens) #float sonuç elde ettik
max_tokens= int(max_tokens)
max_tokens 
#tokenlerin uzunluğunu hespaladık. 

24

In [24]:
np.sum(num_tokens<max_tokens)/len(num_tokens)
#bulduğumuz 59 token uzunluğu toplam tokenlerin yüzde kaçını kapsıyor bulduk %96 az olanlara sıfır ekleyip token sayısını 59 a eşitleyecğiz ger kalan %4 lüük kısmıda silerek 59 token sayısına eşitleyeceğiz

0.9543266667982863

In [25]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)


In [26]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [27]:
x_train_pad.shape

(405208, 24)

In [28]:
x_test_pad.shape

(101302, 24)

In [29]:
np.array(x_train_tokens[800]) #array hali

array([484,  85,  59,   1, 406, 215])

In [30]:
x_train_pad[800] #pad hali

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 484,  85,  59,   1, 406, 215])

In [31]:
idx=tokenizer.word_index

In [32]:
inverse_map = dict(zip(idx.values(),idx.keys()))

In [33]:
def tokens_to_string(tokens):
    words =[inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [34]:
x_train[800]

'аллаһ разы болсын өте тиімді программ а'

In [35]:
tokens_to_string(x_train_tokens[800])

'аллаһ разы болсын өте тиімді а'

In [36]:
model = Sequential() #kerasta model oluşturduk

In [37]:
embedding_size =50  #embedding matirisinin boyutu

In [38]:
#embedding layer ekledik
model.add(Embedding(input_dim=num_words, #Kelime dağarcığı boyutu en başta 10 bin diye verdik
                   output_dim=embedding_size, # Gömülme boyutu kelime vektörü boyutu
                   input_length=max_tokens, #token sayısı
                   name='embedding_layer')) 

In [39]:
model.add(GRU(units=16, return_sequences=True))# true output olarak squences tamamı döndürülür
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [40]:
optimizer =Adam(learning_rate=1e-3) #virgilden sonraki rakam sayısı, adam optimizasyon kütüphasesinden

In [41]:
#modeli derliyoruz
model.compile(loss='binary_crossentropy', #bir modelin tahminlerinin gerçek değerlerden ne kadar uzak olduğunu ölçen bir fonksiyondur.  
             optimizer=optimizer, #optimizsayon algoritması
             metrics=['accuracy'])#modelin başarısı

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 24, 50)            500000    
 )                                                               
                                                                 
 gru (GRU)                   (None, 24, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 24, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504061 (1.92 MB)
Trainable params: 504061 (1.92 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [43]:
model.fit(np.array(x_train_pad), np.array(y_train), epochs=5, batch_size=256) #epochs verilerin kaç defa eğitimden geçeceği, batch_size modeli 256 şar olar verir

Epoch 1/5


1583/1583 [==============================] - 43s 25ms/step - loss: -4.2711 - accuracy: 0.2500
Epoch 2/5
1583/1583 [==============================] - 41s 26ms/step - loss: -10.3723 - accuracy: 0.2500
Epoch 3/5
1583/1583 [==============================] - 41s 26ms/step - loss: -16.4375 - accuracy: 0.2500
Epoch 4/5
1583/1583 [==============================] - 38s 24ms/step - loss: -22.5013 - accuracy: 0.2500
Epoch 5/5
1583/1583 [==============================] - 38s 24ms/step - loss: -28.5741 - accuracy: 0.2500


In [44]:
result=model.evaluate(np.array(x_test_pad), np.array(y_test)) #test seti üzerinde çalıştırıyoruz

3166/3166 [==============================] - 16s 5ms/step - loss: -78.1609 - accuracy: 0.0000e+00


In [45]:
result[1] #isabet oranını görmek için 1 dedik result 2 değer saklar 1. loss değeri 2. isabet oranı (doğruluk)

0.0

In [46]:
result[0] #loss değeri

-78.16087341308594

In [47]:
y_pred = model.predict(x=x_test_pad[0:1000]) # test setinin ilk 1000 vektörü üzerinde tahminde bulundurduk 
y_pred = y_pred.T[0] # modeli vektöre çeviridk vektör üzerinde işlem yapmak kolay olduğu için   .T (transpozu)

32/32 [==============================] - 1s 5ms/step


In [48]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])#

In [49]:
cls_true = np.array(y_test[0:1000])

In [50]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [51]:
len(incorrect)

1000

In [52]:
idx = incorrect[0]
idx

0

In [53]:
text = x_test[idx]
text

'астанада ғана екен ғой'

In [54]:
y_pred[idx]

1.0

In [55]:
cls_true[idx]

2

In [57]:
text1 = "Бұл өнім өте жақсы, мен оны барлығына ұсынамын"
text2 = "Қолданба жылдам болды және мен ешқандай қатып қалуды сезбедім"
text3 = "Бұл өнімде бұл қолданба болмағаны мені қатты ренжітті"
text4 = "мінсіз"
text5 = "Дизайн тамаша, бірақ ойын баяу болып көрінді және іске асыру жылдам болды, мен оны ұсынбаймын"
text6 = "жарнамалық бейнеде көрсетілгендей мүлде емес"
text7 = "Жаман пікірлер мені қорқытты, бірақ менде ешқандай проблема болмады, рахмет"
text8 = "Мен мұндай нашар қолданбаны ешқашан кездестірмедім, мен қолданбаны жоямын"
text9 = "Мен күткендей болды"
text10 = "бұл мен күткендей болмады"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [58]:
tokens = tokenizer.texts_to_sequences(texts)

In [59]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(10, 24)

In [190]:
model.predict(tokens_pad)

1/1 [==============================] - 0s 18ms/step


array([[0.99789584],
       [0.9881163 ],
       [0.15969706],
       [0.996407  ],
       [0.03072071],
       [0.09811385],
       [0.9968926 ],
       [0.00763953],
       [0.99739987],
       [0.34559774]], dtype=float32)